In [1]:
import plotly as py
from plotly import express as px
import folium
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json, requests

In [2]:
def get_json(URL):
  headers = {'Content-type': 'application/json',}
  data = '{}'
  response = requests.post(URL, headers=headers, data=data)
  return(response.text)

r = get_json('https://maphub.net/json/map_load/176607')


#extract incidents
features = json.loads(r)['geojson']['features']
desc = []
for feature in features:
  try:
    desc.append('GROUP: ' + str(feature['properties']['group']) + '\n' + feature['properties']['description'])
  except:
    pass

#extract incident groups and make dictionary
dict_inc_groups = {str(group['id']) : str(group['title']) for group in json.loads(r)['geojson']['groups']}

dict_inc_groups

{'1204076368': 'Russian Firing Positions',
 '1302000987': 'Civilian Casualty',
 '1422527994': 'Russian Military Movements February 2022',
 '182598014': 'Civilian Infrastructure Damage February 2022',
 '1946403459': 'Other',
 '2148921144': 'Civilian Infrastructure Damage April 2022',
 '2159062771': 'Civilian Infrastructure Damage March 2022',
 '2327490580': 'Russian Military Movements January 2022',
 '2613895895': 'Gunfire, fighting, battle',
 '2758805904': 'Ukrainian Military Losses',
 '2826993048': 'Russian Military Movements April 2022',
 '3442810439': 'Military Infrastructure Damage',
 '3640522656': 'Russian Allies Movements/Losses',
 '3730511149': 'Munitions',
 '3840102509': 'Border Lines',
 '4263226937': 'Russian Military Movements March 2022',
 '744433847': 'Bombing, shelling or explosion',
 '878638621': 'Russian Military Losses'}

In [3]:
#make dataframe
df_inc = pd.DataFrame(columns=[data.split(':',1)[0] for data in desc[0].split('\n')])
df_inc

#create dictionary
dict_inc = {}

#iterate through raw data and append to dataframe as dictionary
for entry in desc:
  for item in entry.split('\n'):
    try:
      key, value = item.split(':',1)
      dict_inc[key] = value.lstrip()
    except:
      dict_inc[key] = ''
  df_inc = df_inc.append(dict_inc, ignore_index=True)
  dict_inc = {}

df_inc = df_inc.fillna('')
df_inc['GROUP'] = df_inc['GROUP'].map(dict_inc_groups)
df_inc.to_csv('/content/drive/MyDrive/Colab Notebooks/Ukraine Analysis and Data/Russia-Ukraine_Monitor_Map.csv')

In [4]:
#load incident as data frame
df_inc.drop(df_inc[df_inc['COORDINATES'] == ''].index,inplace = True)
df_inc.reset_index(drop=True, inplace=True)
df_inc[['lat','lon']] = df_inc.COORDINATES.str.split("," , expand=True)
df_inc[['lat','lon']] = df_inc[['lat','lon']].apply(pd.to_numeric)

In [5]:
df_inc2 = df_inc.iloc[df_inc[(df_inc['GROUP'].str.contains("Infrastructure Damage")) | (df_inc['GROUP'].str.contains("Bombing")) | (df_inc['GROUP'].str.contains("Gunfire")) | (df_inc['GROUP'].str.contains("Munitions"))].index]
set(df_inc2['GROUP'])
#df_inc2 = df_inc.drop(df_inc[(df_inc['GROUP'].str.contains("Russian") == True) | (df_inc['GROUP'].str.contains("Other") == True)].index)

{'Bombing, shelling or explosion',
 'Civilian Infrastructure Damage April 2022',
 'Civilian Infrastructure Damage February 2022',
 'Civilian Infrastructure Damage March 2022',
 'Gunfire, fighting, battle',
 'Military Infrastructure Damage',
 'Munitions'}

In [6]:
fig = px.scatter_mapbox(df_inc2, lat = df_inc2['lat'], lon = df_inc2['lon'],zoom = 3, width = 1600, height = 1200, hover_data = ["GROUP","BRIEF DESCRIPTION"], color = "VIOLENCE LEVEL")
fig.update_layout(mapbox_style='open-street-map')
fig.show()